# Titiler expertiments
This notebook provides two possible ways to work with titiler:
- first each dataset is a separate cog thus a diferent cog endpoint
- one fat dataset with each raster as a band so we can use to `expression` in the titiler to filter on by another

In [3]:
import json
from pathlib import Path

import requests
import httpx
import branca
from folium import Map, LayerControl
from folium.raster_layers import TileLayer

titiler_endpoint = "http://127.0.0.1:8083"

In [4]:
def make_expression(filter_by: list=None, no_data=-9999):
    """Build a where expression to filters foodscapes"""
    if filter_by:
        filter_parts = "|".join([f"(b8=={x})" for x in filter_by])
        return f"where({filter_parts}, b8, {no_data});"  # the trailing ";" is critical to make it work
    else:
        return ""

def filter_with_colormap(values: list[int], cmap: dict) -> dict:
    """Changes alpha to 0 of all values in cmap except the ones in values

    Can be used as an alternative way to filter
    
    *NEVER USE!*
    """
    new_cmap = {}
    values = [str(v) for v in values]
    for k, v in cmap.items():
        if k in values:
            if len(v)==7:  # no alpha so add the alfa byte
                v = v + "FF"
            else:
                v[-2:]= "FF"
        else:
            if len(v)==7:
                v = v + "00"
            else:
                v[-2:]= "00"
        new_cmap[k] = v
    return new_cmap


def show_map(tiles):
    fig = branca.element.Figure(width="800px", height="600px")
    m = Map()
    fig.add_child(m)

    layer = TileLayer(
        tiles,
        attr="TNC",
        name="fs",
        overlay=True
    )
    m.add_child(layer)
    LayerControl().add_to(m)
    return m
    

with open("foodscapes_cmap.json") as f:
    foodscpaes_cmap = json.load(f)
    # foodscpaes_cmap = filter_with_colormap([707], foodscpaes_cmap)

## Simple cog

One COG per layer, multiple endpoints

__DON'T USE__ can't filter one raster by the value of another or mask

In [5]:
# fs_url = Path("/home/biel/Vizzuality/dev/titilerTest/demo/foodscapes/cog_tiler/cogs/Foodscapes_combinedGEOTIFF_final_COG.tif")
# paths if titiler is up with docker compose
fs_url = Path("/data/Foodscapes_combinedGEOTIFF_final_COG.tif")
risk_url = Path("/data/biodiversityrisk_COG.tif")
intensity_url = Path("/data/Foodscapes_combinedGEOTIFF_intensity_COG.tif")

In [4]:


filter_by = list(range(701, 720))
# filter_by = None

fs_r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "tile_format": "png",
        "url": fs_url,
        "colormap":json.dumps(foodscpaes_cmap),
        "return_mask":"true",
        "expression": make_expression(filter_by, -9999)  # foodscapes cog nodata value
    }
).json()

risk_r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "tile_format": "png",
        "url": risk_url,
        "colormap_name": "viridis_r",
        "expression": "where(b1>=0, b1, 0);"
    }
).json()

intensity_r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params={
        "tile_format": "png",
        "url": intensity_url,
        "colormap_name": "viridis",
        "rescale": "0,3"
    }
).json()

In [5]:
fig = branca.element.Figure(width="800px", height="600px")
m = Map()
fig.add_child(m)

fs_layer = TileLayer(
    fs_r['tiles'][0],
    attr="TNC",
    name="fs",
    overlay=True
)

risk_layer = TileLayer(
        risk_r["tiles"][0],
        attr="TNC",
        name="risk",
        overlay=True
)

intensity_layer = TileLayer(
        intensity_r["tiles"][0],
        attr="TNC",
        name="intensity",
        overlay=True
)

m.add_child(fs_layer)
m.add_child(risk_layer)
m.add_child(intensity_layer)

LayerControl().add_to(m)
m

## One single COG, multiple bands

In [26]:
# path maped into docker compose cogs/ -> /data/
stack_url = "/data/all.tif"

In [30]:
# b1: biodiversity risk
# b2: foodscapes
# b3: intensity

# this filters foodscapes by medium intensity
expression = "where(b3==3, b2, -9999);"

r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "tile_format": "png",
        "url": stack_url,
        "colormap":json.dumps(foodscpaes_cmap),
        # "expression": expression,
        "bidx":5
    }
).json()

tiles = r["tiles"][0]
show_map(tiles)

## One single COG, all Risk + Foodscape bands

In [90]:
cog_url = "/data/foodscapes_stack_cog_lzw.tif"
# expression = "where(b8==709, b5, -9999);"
expression = make_expression(list(range(500, 700)))
expression = "where((b8>100) & (b8<700), b8, -9999);"

risk_cmap = {
    -9999: "#00000000",
    1: "#FF2A00",
    2: "#FF0000",
    3: "#00FF00"
}
foodscpaes_cmap[1] = "#FF2A0000"

r = httpx.get(
    f"{titiler_endpoint}/tilejson.json",
    params = {
        "tile_format": "png",
        "url": cog_url,
        "colormap":json.dumps(foodscpaes_cmap),
        # "colormap":json.dumps(risk_cmap),
        # "colormap_name":"viridis",
        # "nodata":-9999,
        # "expression": expression,
        "bidx": 8
    }
).json()

tiles = r["tiles"][0]
show_map(tiles)

In [91]:
%%time
with open("/home/biel/Vizzuality/foodscapes/data/east_kalimantan.geojson") as f:
    geom = json.load(f)

r = httpx.post(
    f"{titiler_endpoint}/statistics",
    content=json.dumps(geom),
    params = {
        "url": cog_url,
        "bidx": [8,7],
        "categorical": True
    }
)

CPU times: user 271 ms, sys: 0 ns, total: 271 ms
Wall time: 3.77 s


In [92]:
import geopandas as gpd

In [93]:
df = gpd.read_file(json.dumps(r.json()))

In [94]:
df.statistics[0]

{'b8': {'min': -9999.0,
  'max': 1108.0,
  'mean': 183.20926807160663,
  'count': 8826.0,
  'sum': 1617005.0,
  'std': 1727.6859733173949,
  'median': 1.0,
  'majority': 1.0,
  'minority': 1002.0,
  'unique': 16.0,
  'histogram': [[226.0,
    4877.0,
    6.0,
    6.0,
    230.0,
    40.0,
    1.0,
    477.0,
    1240.0,
    29.0,
    12.0,
    139.0,
    1382.0,
    7.0,
    142.0,
    12.0],
   [-9999.0,
    1.0,
    304.0,
    603.0,
    902.0,
    906.0,
    1002.0,
    1003.0,
    1004.0,
    1005.0,
    1007.0,
    1102.0,
    1104.0,
    1106.0,
    1107.0,
    1108.0]],
  'valid_percent': 39.23,
  'masked_pixels': 13670.0,
  'valid_pixels': 8826.0,
  'percentile_2': -9999.0,
  'percentile_98': 1104.0},
 'b7': {'min': -9999.0,
  'max': 1.0,
  'mean': -5819.303648311806,
  'count': 8826.0,
  'sum': -51361174.0,
  'std': 4932.25120250037,
  'median': -9999.0,
  'majority': -9999.0,
  'minority': 1.0,
  'unique': 2.0,
  'histogram': [[5137.0, 3689.0], [-9999.0, 1.0]],
  'valid_perce